# OpenRemote: Energy Transition

## Intro

### Technologies used

## Imports

In [87]:
import warnings
import datetime as dt
import itertools
import pandas as pd
from pytz import timezone
import numpy as np
import matplotlib.pyplot as plt

In [88]:
PATH = ".\\data\\"

In [89]:
!dir {PATH} #Windows Only Command

 Volume in drive C is Windows
 Volume Serial Number is 984A-EC2B

 Directory of C:\Users\simon\OneDrive - PXL\School\2TINQ\_Research Project\OpenRemote-Energy-Challenge\machine-learning\data

02/05/2021  19:20    <DIR>          .
02/05/2021  19:20    <DIR>          ..
28/03/2021  20:31            13.803 GetWeatherSiteEstimatedActuals.csv
12/04/2021  10:27            20.994 GetWeatherSiteEstimatedActuals2.csv
23/03/2021  20:55           340.288 SolarData.csv
               3 File(s)        375.085 bytes

 Directory of C:\Users\simon\OneDrive - PXL\School\2TINQ\_Research Project\OpenRemote-Energy-Challenge\machine-learning


 Directory of C:\Users\simon\OneDrive - PXL\School\2TINQ\_Research Project\OpenRemote-Energy-Challenge\machine-learning


 Directory of C:\Users\simon\OneDrive - PXL\School\2TINQ\_Research Project\OpenRemote-Energy-Challenge\machine-learning



File Not Found


In [90]:
#Importing datasets
solar_data = pd.read_csv(f"{PATH}SolarData.csv", parse_dates=["timestamp"])
solcast_predictions = pd.read_csv(f"{PATH}GetWeatherSiteEstimatedActuals2.csv", parse_dates=["period_end"])

## Handy functions

In [91]:
def display_all(df):
    with pd.option_context("display.max_rows", 10000, "display.max_columns", 10000): 
        display(df)

## Machine learning

### Variables

In [92]:
PREDICTION_PERIOD = 30 #Minutes
TIME_ZONE = timezone("Europe/Amsterdam")

### Cleaning the data

In [93]:
solar_data #OpenRemote Data

,timestamp,name,attribute_name,value
0,2021-02-11 04:10:22.926,Tamar building,totalPower,0.0
1,2021-02-11 04:10:23.089,Humber building,totalPower,0.0
2,2021-02-25 04:13:48.669,Humber building,totalPower,0.0
3,2021-02-25 04:13:54.488,Tamar building,totalPower,0.0
4,2021-02-11 04:40:23.122,Tamar building,totalPower,0.0
...,...,...,...,...
5469,2021-02-25 02:43:54.267,Tamar building,totalPower,0.0
5470,2021-02-25 03:13:48.480,Humber building,totalPower,0.0
5471,2021-02-25 03:13:54.344,Tamar building,totalPower,0.0
5472,2021-02-25 03:43:48.586,Humber building,totalPower,0.0


**Global Horizontal Irradiance** (`Ghi`) is the total solar radiation on a horizontal surface. It is the sum of DNI & DHI  
**Direct Normal Irradiance** (`Dni`) is the amount of solar radiation received per unit area by a surface that is always held in a straight line from the sun's current position  
**Diffuse Horizontal Irradiance** (`Dhi`) is the radiation at the Earth's surface from light scattered by the atmosphere.  
**Direct Beam Horizontal Irradiance** (`Ebh`) The horizontal component of DNI  
**Zenith** is the angle between the sun's rays and the vertical direction.  
**Azimuth** is the angle between the projection of sun's center onto the horizontal plane and due south direction.  
  
*10 and 90 fields represent a 10% and 90% probability bound, around a median forecast value*

In [94]:
solcast_predictions #Data from solcast

,Ghi,Ghi90,Ghi10,Ebh,Dni,Dni10,Dni90,Dhi,air_temp,Zenith,Azimuth,cloud_opacity,period_end,Period
0,294,374,141,82,167,56,475,211,5,60,-117,40,2021-04-12 08:30:00+00:00,PT30M
1,184,558,25,4,8,64,860,180,6,56,-125,67,2021-04-12 09:00:00+00:00,PT30M
2,339,561,133,44,73,3,636,295,6,52,-133,46,2021-04-12 09:30:00+00:00,PT30M
3,561,677,68,317,487,1,887,244,8,49,-141,17,2021-04-12 10:00:00+00:00,PT30M
4,644,718,190,448,662,9,913,196,8,47,-151,10,2021-04-12 10:30:00+00:00,PT30M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,75,114,5,9,58,0,341,66,8,81,-85,34,2021-04-19 06:00:00+00:00,PT30M
332,128,193,9,18,78,0,475,108,8,76,-91,34,2021-04-19 06:30:00+00:00,PT30M
333,184,279,13,29,92,0,565,154,8,71,-97,34,2021-04-19 07:00:00+00:00,PT30M
334,239,362,17,44,113,0,682,196,8,67,-103,34,2021-04-19 07:30:00+00:00,PT30M


In [95]:
totalPower = solar_data.loc[solar_data['attribute_name'] == 'totalPower'].sort_values(by=['timestamp'], ascending=True).drop(columns=['attribute_name'])

In [96]:
totalPower

,timestamp,name,value
749,2021-02-05 01:08:59.353,Humber building,0.000000
750,2021-02-05 01:08:59.592,Tamar building,0.000000
767,2021-02-05 01:38:59.518,Humber building,0.000000
776,2021-02-05 01:38:59.678,Tamar building,0.000000
811,2021-02-05 02:08:59.722,Humber building,0.000000
...,...,...,...
3247,2021-03-08 11:16:29.019,Tamar building,12.726667
3256,2021-03-08 11:46:28.730,Humber building,2.689000
3257,2021-03-08 11:46:29.236,Tamar building,11.336000
3272,2021-03-08 12:16:28.843,Humber building,3.391500


In [97]:
humber_totalPower = totalPower.loc[totalPower['name'] == 'Humber building'].drop(columns=['name'])
tamar_totalPower = totalPower.loc[totalPower['name'] == 'Tamar building'].drop(columns=['name'])

In [98]:
tamar_totalPower, humber_totalPower

(                   timestamp      value
 750  2021-02-05 01:08:59.592   0.000000
 776  2021-02-05 01:38:59.678   0.000000
 812  2021-02-05 02:09:00.184   0.000000
 816  2021-02-05 02:39:00.276   0.000000
 839  2021-02-05 03:09:00.357   0.000000
 ...                      ...        ...
 3221 2021-03-08 10:16:28.661   6.579666
 3235 2021-03-08 10:46:28.926  11.877333
 3247 2021-03-08 11:16:29.019  12.726667
 3257 2021-03-08 11:46:29.236  11.336000
 3273 2021-03-08 12:16:29.333  15.811334
 
 [1465 rows x 2 columns],
                    timestamp     value
 749  2021-02-05 01:08:59.353  0.000000
 767  2021-02-05 01:38:59.518  0.000000
 811  2021-02-05 02:08:59.722  0.000000
 815  2021-02-05 02:38:59.931  0.000000
 840  2021-02-05 03:09:00.400  0.000000
 ...                      ...       ...
 3220 2021-03-08 10:16:27.919  1.498000
 3234 2021-03-08 10:46:28.337  2.603000
 3246 2021-03-08 11:16:28.467  2.955333
 3256 2021-03-08 11:46:28.730  2.689000
 3272 2021-03-08 12:16:28.843  3.391500


In [99]:
totalPower

,timestamp,name,value
749,2021-02-05 01:08:59.353,Humber building,0.000000
750,2021-02-05 01:08:59.592,Tamar building,0.000000
767,2021-02-05 01:38:59.518,Humber building,0.000000
776,2021-02-05 01:38:59.678,Tamar building,0.000000
811,2021-02-05 02:08:59.722,Humber building,0.000000
...,...,...,...
3247,2021-03-08 11:16:29.019,Tamar building,12.726667
3256,2021-03-08 11:46:28.730,Humber building,2.689000
3257,2021-03-08 11:46:29.236,Tamar building,11.336000
3272,2021-03-08 12:16:28.843,Humber building,3.391500


In [100]:
solar_predictions = solcast_predictions.drop(columns=['Ghi90', 'Ghi10', 'Dni90', 'Dni10'])

In [101]:
solar_predictions

,Ghi,Ebh,Dni,Dhi,air_temp,Zenith,Azimuth,cloud_opacity,period_end,Period
0,294,82,167,211,5,60,-117,40,2021-04-12 08:30:00+00:00,PT30M
1,184,4,8,180,6,56,-125,67,2021-04-12 09:00:00+00:00,PT30M
2,339,44,73,295,6,52,-133,46,2021-04-12 09:30:00+00:00,PT30M
3,561,317,487,244,8,49,-141,17,2021-04-12 10:00:00+00:00,PT30M
4,644,448,662,196,8,47,-151,10,2021-04-12 10:30:00+00:00,PT30M
...,...,...,...,...,...,...,...,...,...,...
331,75,9,58,66,8,81,-85,34,2021-04-19 06:00:00+00:00,PT30M
332,128,18,78,108,8,76,-91,34,2021-04-19 06:30:00+00:00,PT30M
333,184,29,92,154,8,71,-97,34,2021-04-19 07:00:00+00:00,PT30M
334,239,44,113,196,8,67,-103,34,2021-04-19 07:30:00+00:00,PT30M


In [102]:
solar_predictions.period_end

0      2021-04-12 08:30:00+00:00
1      2021-04-12 09:00:00+00:00
2      2021-04-12 09:30:00+00:00
3      2021-04-12 10:00:00+00:00
4      2021-04-12 10:30:00+00:00
                 ...            
331    2021-04-19 06:00:00+00:00
332    2021-04-19 06:30:00+00:00
333    2021-04-19 07:00:00+00:00
334    2021-04-19 07:30:00+00:00
335    2021-04-19 08:00:00+00:00
Name: period_end, Length: 336, dtype: object

In [103]:
solar_predictions['period_end'] = pd.DatetimeIndex(pd.to_datetime(solar_predictions.period_end, utc=True)).tz_convert(TIME_ZONE)

In [104]:
solar_predictions['period_start'] = solar_predictions['period_end'] - pd.Timedelta(minutes=PREDICTION_PERIOD)
solar_predictions = solar_predictions.drop(columns=['Period'])

In [105]:
totalPower['timestamp'] = totalPower['timestamp'].dt.tz_localize(TIME_ZONE)

In [110]:
# Cleaned DataFrames:
solar_predictions#, totalPower

,Ghi,Ebh,Dni,Dhi,air_temp,Zenith,Azimuth,cloud_opacity,period_end,period_start
0,294,82,167,211,5,60,-117,40,2021-04-12 10:30:00+02:00,2021-04-12 10:00:00+02:00
1,184,4,8,180,6,56,-125,67,2021-04-12 11:00:00+02:00,2021-04-12 10:30:00+02:00
2,339,44,73,295,6,52,-133,46,2021-04-12 11:30:00+02:00,2021-04-12 11:00:00+02:00
3,561,317,487,244,8,49,-141,17,2021-04-12 12:00:00+02:00,2021-04-12 11:30:00+02:00
4,644,448,662,196,8,47,-151,10,2021-04-12 12:30:00+02:00,2021-04-12 12:00:00+02:00
...,...,...,...,...,...,...,...,...,...,...
331,75,9,58,66,8,81,-85,34,2021-04-19 08:00:00+02:00,2021-04-19 07:30:00+02:00
332,128,18,78,108,8,76,-91,34,2021-04-19 08:30:00+02:00,2021-04-19 08:00:00+02:00
333,184,29,92,154,8,71,-97,34,2021-04-19 09:00:00+02:00,2021-04-19 08:30:00+02:00
334,239,44,113,196,8,67,-103,34,2021-04-19 09:30:00+02:00,2021-04-19 09:00:00+02:00


In [117]:
#Test loop - Full loop could take +- 10 mins to run!
totalPower_ghi = []
totalPower_ebh = []
totalPower_dni = []
totalPower_dhi = []
totalPower_air_temp = []
totalPower_zenith = []
totalPower_azimuth = []
totalPower_cloud_opacity = []

for i, totalPower_row in totalPower.head().iterrows():
    for j, prediction_row in solar_predictions.head().iterrows():
        if totalPower_row.timestamp > prediction_row.period_start and totalPower_row.timestamp < prediciton_row.period_end:
            print(i, j, totalPower_row.timestamp, "is between", prediction_row.period_start, "&", prediction_row.period_end)
            totalPower_ghi.append(solar_predictions[j]['Ghi'])
            totalPower_ebh.append(solar_predictions[j]['Ebh'])
            totalPower_dni.append(solar_predictions[j]['Dni'])
            totalPower_dhi.append(solar_predictions[j]['Dhi'])
            totalPower_air_temp.append(solar_predictions[j]['air_temp'])
            totalPower_zenith.append(solar_predictions[j]['Zenith'])
            totalPower_azimuth.append(solar_predictions[j]['Azimuth'])
            totalPower_cloud_opacity.append(solar_predictions[j]['cloud_opacity'])
        else:
            print(i, j, "No matching dates")
            
totalPower['Ghi'] = pd.Series(totalPower_ghi, dtype='Int64')
totalPower['Ebh'] = pd.Series(totalPower_ebh, dtype='Int64')
totalPower['Dni'] = pd.Series(totalPower_dni, dtype='Int64')
totalPower['Dhi'] = pd.Series(totalPower_dhi, dtype='Int64')
totalPower['air_temp'] = pd.Series(totalPower_air_temp, dtype='Int64')
totalPower['Zenith'] = pd.Series(totalPower_zenith, dtype='Int64')
totalPower['Azimuth'] = pd.Series(totalPower_azimuth, dtype='Int64')
totalPower['cloud_opacity'] = pd.Series(totalPower_cloud_opacity, dtype='Int64')

749 0 No matching dates
749 1 No matching dates
749 2 No matching dates
749 3 No matching dates
749 4 No matching dates
750 0 No matching dates
750 1 No matching dates
750 2 No matching dates
750 3 No matching dates
750 4 No matching dates
767 0 No matching dates
767 1 No matching dates
767 2 No matching dates
767 3 No matching dates
767 4 No matching dates
776 0 No matching dates
776 1 No matching dates
776 2 No matching dates
776 3 No matching dates
776 4 No matching dates
811 0 No matching dates
811 1 No matching dates
811 2 No matching dates
811 3 No matching dates
811 4 No matching dates
